In [103]:
import pandas as pd
import matplotlib.pyplot as plt

import nltk
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from nltk.stem import PorterStemmer
from sklearn.pipeline import Pipeline

In [84]:
df = pd.read_csv('train1.csv')
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [85]:
df.shape

(27480, 4)

In [86]:
# Filtre le DataFrame pr éliminer les lignes où 'neutral' apparait : 
df_filtered = df[df['sentiment'] != 'neutral']

df_filtered

,textID,text,selected_text,sentiment
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive
...,...,...,...,...
27474,b78ec00df5,enjoy ur night,enjoy,positive
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive


In [87]:
df_filtered.shape

(16363, 4)

In [88]:
# Calcule la fréqce des sentimts :
sentiment = df_filtered['sentiment'].value_counts()

# Calcule le % de chaq sentimt :
sentiment_pourcent = (sentiment / sentiment.sum()) * 100

sentiment_pourcent

sentiment
positive    52.447595
negative    47.552405
Name: count, dtype: float64

In [89]:
# Téléchrge les ressrces nécessaires :
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialise le lemmatiseur et les stopwords :
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('french'))
punctuation = set(string.punctuation)

# ¨Prétraitemt du texte :
def clean(text):
    """
    Fonction de nettoyage d'un texte : 
    - Tokenisation
    - Conversion en minuscules
    - Suppression des stopwords et de la ponctuation
    - Application du lemmatiseur

    :param text: Texte à nettoyer (str)
    :return: Texte nettoyé sous forme de tokens (str)
    """

    # Tokenisat° :
    tokens = word_tokenize(text)
    
    # Cvers° en minuscules et suppress° des mots vides :
    tokens = [token.lower() for token in tokens if token.isalpha() and token.lower() not in stop_words]
    
    # Recposit° du texte nettoyé ss frme de chaîne avec des espaces entre les tokens :
    cleaned_text = " ".join(tokens)
    
    return cleaned_text

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sebla\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sebla\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sebla\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [90]:
# Test :
texte = "You are better when I am well !"
texte_nettoye = clean(texte)

print("Texte original:")
print(texte)

print("\nTexte nettoyé (lemmatisé):")
print(texte_nettoye)

Texte original:
You are better when I am well !

Texte nettoyé (lemmatisé):
you are better when i am well


In [91]:
# Initialise du stemmer :
stemmer = PorterStemmer()

def clean_stemmer(text):
    """
    Fonction de nettoyage d'un texte avec un stemmer :
    - Tokenisation
    - Conversion en minuscules
    - Suppression des stopwords et de la ponctuation
    - Application du stemmer

    :param text: Texte à nettoyer (str)
    :return: Texte nettoyé sous forme de tokens (str)
    """
    # Tokenisat° des mots :
    tokens = word_tokenize(text)
    
    # Prétraitemt des tokens : suppress° des mots vides, ponctuat° et convers° en minuscules :
    tokens = [stemmer.stem(token.lower()) for token in tokens if token.isalpha() and token.lower() not in stop_words]
    
    # Recposit° du texte nettoyé ss frme de chaîne avec des espaces entre les tokens :
    cleaned_text = " ".join(tokens)
    
    return cleaned_text

In [92]:
# Test :
texte = "You are better when I am well !"
texte_nettoye = clean(texte)

print("Texte original:")
print(texte)

print("\nTexte nettoyé (stemmer):")
print(texte_nettoye)

Texte original:
You are better when I am well !

Texte nettoyé (stemmer):
you are better when i am well


In [93]:
# Initialise le lemmatiseur :
lemmatizer = WordNetLemmatizer()

# Récupére les stopwords anglais :
stopwordsenglish = set(stopwords.words('english'))

# Ponctuat° à exclure :
punctuation = set(string.punctuation)

def clean(text):
    """
    Fonction de nettoyage d'un texte :
    - Tokenisation
    - Conversion en minuscules
    - Suppression des stopwords en français et anglais
    - Suppression de la ponctuation
    - Application du lemmatiseur

    :param text: Texte à nettoyer (str)
    :return: Texte nettoyé sous forme de tokens (str)
    """
    # Tokenisat° des mots :
    tokens = word_tokenize(text)
    
    # Prétraitemt des tokens :
    # - Suppress° des mots vides (stopwords en anglais et en français)
    # - Suppress° des signes de ponctuat°
    # - Convers° en minuscules et lemmatisat°
    tokens = [
        lemmatizer.lemmatize(token.lower()) 
        for token in tokens 
        if token.isalpha() and token.lower() not in stopwordsenglish and token.lower() not in stopwords.words('french') and token not in punctuation
    ]
    
    # Recpose le texte nettoyé ss frme de chaîne avec des espaces entre les tokens :
    cleaned_text = " ".join(tokens)
    
    return cleaned_text

In [94]:
# Test :
texte = "The children are eating apples, and they are playing in the park!"

# Appliq la fct° clean :
texte_nettoye = clean(texte)

print("Texte original:")
print(texte)

print("\nTexte nettoyé:")
print(texte_nettoye)

Texte original:
The children are eating apples, and they are playing in the park!

Texte nettoyé:
child eating apple playing park


In [96]:
# Appliq la fct° 'clean' à chaq élémt de la colonne 'text'
df['clean'] = df['text'].apply(clean)

# Affiche le DataFrame avec la nvelle colonne 'clean' :
df[['text', 'clean']].head()

,text,clean
0,"I`d have responded, if I were going",responded going
1,Sooo SAD I will miss you here in San Diego!!!,sooo sad miss san diego
2,my boss is bullying me...,bos bullying
3,what interview! leave me alone,interview leave alone
4,"Sons of ****, why couldn`t they put them on t...",son put release already bought


In [97]:
# Copie la colonne 'clean' ds 1 série X (caractéristiq d'entrée) :
X = df['clean']

# Copie la colonne 'sentimt' ds 1 série y (labels de sortie) :
y = df['sentiment']

# Appliq train-test split :
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=32)

print("Taille de l'ensemble d'entraînement (X_train, y_train):", len(X_train))
print("Taille de l'ensemble de test (X_test, y_test):", len(X_test))

Taille de l'ensemble d'entraînement (X_train, y_train): 20610
Taille de l'ensemble de test (X_test, y_test): 6870


In [106]:
# Créer 1 mdèle CountVectorizer :
vectorizer = CountVectorizer()

# Entraîne le mdèle sur X_train et créer la matrice de caractéristiq X_train_CV :
X_train_CV = vectorizer.fit_transform(X_train)

# Créer la matrice de caractéristiq X_test_CV en utilisant le même mdèle sans réentraîner :
X_test_CV = vectorizer.transform(X_test)

# Entraîner 1 mdèle de régress° logistiq sur les données vectorisées
model = LogisticRegression(max_iter=1000)
model.fit(X_train_CV, y_train)

# Prédis sur l'ensble de test :
y_pred_CV = model.predict(X_test_CV)

# Évaluat° du mdèle :
accuracy_CV = accuracy_score(y_test, y_pred_CV)
print(f"Accuracy du modèle : {accuracy:.4f}")

Accuracy du modèle : 0.6902


In [99]:
# Créer 1 mdèle TfidfVectorizer :
tfidf_vectorizer = TfidfVectorizer()

# Entraîne le mdèle sur X_train et créer la matrice de caractéristiq X_train_CV :
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Créer la matrice de caractéristiq X_test_CV en utilisant le même mdèle sans réentraîner :
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Entraîner 1 mdèle de régress° logistiq sur les données vectorisées
model_tfidf = LogisticRegression(max_iter=1000)
model_tfidf.fit(X_train_tfidf, y_train)

# Prédis sur l'ensble de test :
y_pred_tfidf = model.predict(X_test_tfidf)

# Évaluat° du mdèle :
accuracy_tfidf = accuracy_score(y_test, y_pred_tfidf)
print(f"Accuracy du modèle : {accuracy:.4f}")

# Affiche des résultats de cparais° :
print("Résultats pour CountVectorizer :")
print(f"Accuracy (CountVectorizer) : {accuracy_CV:.4f}")
print("\nClassification Report (CountVectorizer) :")
print(classification_report(y_test, y_pred_CV))

print("\nRésultats pour TfidfVectorizer :")
print(f"Accuracy (TfidfVectorizer) : {accuracy_tfidf:.4f}")
print("\nClassification Report (TfidfVectorizer) :")
print(classification_report(y_test, y_pred_tfidf))

Accuracy du modèle : 0.6902
Résultats pour CountVectorizer :
Accuracy (CountVectorizer) : 0.6902

Classification Report (CountVectorizer) :
              precision    recall  f1-score   support

    negative       0.70      0.61      0.65      1944
     neutral       0.64      0.72      0.68      2772
    positive       0.75      0.72      0.74      2154

    accuracy                           0.69      6870
   macro avg       0.70      0.68      0.69      6870
weighted avg       0.69      0.69      0.69      6870


Résultats pour TfidfVectorizer :
Accuracy (TfidfVectorizer) : 0.5622

Classification Report (TfidfVectorizer) :
              precision    recall  f1-score   support

    negative       0.82      0.32      0.46      1944
     neutral       0.48      0.94      0.64      2772
    positive       0.90      0.29      0.44      2154

    accuracy                           0.56      6870
   macro avg       0.73      0.52      0.51      6870
weighted avg       0.71      0.56      0

In [ ]:
# Les meilleurs paramètres sont obtenus avec le modèle de CountVectorizer.

In [105]:
# Séparat° des données en jeu d'entraînemt et de test :
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=32)

# Pramètres à tester pr CountVectorizer et TfidfVectorizer :
param_grid_cv = {
    'vectorizer__max_df': [0.75, 1.0],  # Ignore les mots apparaissant ds + de 75% des documts
    'vectorizer__min_df': [1, 5],  # Ignore les mots apparaissant ds - de 5 documts
    'vectorizer__ngram_range': [(1, 1), (1, 2)],  # Unigrammes ou Bigrammes
    'vectorizer__stop_words': ['english', None]  # Utilise les stopwords anglais ou pas
}

param_grid_tfidf = {
    'vectorizer__max_df': [0.75, 1.0],
    'vectorizer__min_df': [1, 5],
    'vectorizer__ngram_range': [(1, 1), (1, 2)],
    'vectorizer__stop_words': ['english', None]
}

# Pipeline avec CountVectorizer :
count_vectorizer = CountVectorizer()

# Créat° du Pipeline pr CountVectorizer :
pipeline_cv = Pipeline([
    ('vectorizer', count_vectorizer),
    ('classifier', LogisticRegression(max_iter=1000))  # Régress° logistiq
])

# GridSearchCV pr CountVectorizer avec Régress° logistiq :
grid_search_cv = GridSearchCV(
    estimator=pipeline_cv, 
    param_grid=param_grid_cv, 
    cv=5,  # 5-fold Cross-validat°
    scoring='accuracy', 
    n_jobs=-1,  # Utiliser ts les cœurs de CPU
    verbose=1
)

# Apprentissage avec GridSearchCV pr CountVectorizer :
grid_search_cv.fit(X_train, y_train)

# Affiche les meilleurs pramètres trvés :
print("Meilleurs pramètres pr CountVectorizer :")
print(grid_search_cv.best_params_)
print(f"Meilleur score de validation : {grid_search_cv.best_score_:.4f}")

# Pipeline avec TfidfVectorizer :
tfidf_vectorizer = TfidfVectorizer()

# Créat° du Pipeline pr TfidfVectorizer :
pipeline_tfidf = Pipeline([
    ('vectorizer', tfidf_vectorizer),
    ('classifier', LogisticRegression(max_iter=1000))  # Régress° logistiq
])

# GridSearchCV pr TfidfVectorizer avec Régress° logistiq
grid_search_tfidf = GridSearchCV(
    estimator=pipeline_tfidf, 
    param_grid=param_grid_tfidf, 
    cv=5, 
    scoring='accuracy', 
    n_jobs=-1,
    verbose=1
)

# Apprentissage avec GridSearchCV pr TfidfVectorizer :
grid_search_tfidf.fit(X_train, y_train)

# Affiche les meilleurs pramètres trvés :
print("\nMeilleurs paramètres pour TfidfVectorizer :")
print(grid_search_tfidf.best_params_)
print(f"Meilleur score de validation : {grid_search_tfidf.best_score_:.4f}")

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Meilleurs pramètres pr CountVectorizer :
{'vectorizer__max_df': 0.75, 'vectorizer__min_df': 1, 'vectorizer__ngram_range': (1, 2), 'vectorizer__stop_words': 'english'}
Meilleur score de validation : 0.6923
Fitting 5 folds for each of 16 candidates, totalling 80 fits

Meilleurs paramètres pour TfidfVectorizer :
{'vectorizer__max_df': 0.75, 'vectorizer__min_df': 1, 'vectorizer__ngram_range': (1, 2), 'vectorizer__stop_words': 'english'}
Meilleur score de validation : 0.6882


In [81]:
# Cross-validat° pr évaluer les mdèles afin de trver les meilleurs pramètres :
# Cross-validat° pr CountVectorizer :
cv_score_cv = cross_val_score(
    LogisticRegression(max_iter=1000), 
    count_vectorizer.fit_transform(X_train), 
    y_train, 
    cv=5, 
    scoring='accuracy'
)

print("\nCross-validation Scores pour CountVectorizer :")
print(cv_score_cv)
print(f"Moyenne Cross-validation : {cv_score_cv.mean():.4f}")


# Cross-validat° pr TfidfVectorizer :
cv_score_tfidf = cross_val_score(
    LogisticRegression(max_iter=1000), 
    tfidf_vectorizer.fit_transform(X_train), 
    y_train, 
    cv=5, 
    scoring='accuracy'
)

print("\nCross-validation Scores pour TfidfVectorizer :")
print(cv_score_tfidf)
print(f"Moyenne Cross-validation : {cv_score_tfidf.mean():.4f}")


Cross-validation Scores pour CountVectorizer :
[0.68558952 0.69311014 0.6768559  0.69335274 0.68073751]
Moyenne Cross-validation : 0.6859

Cross-validation Scores pour TfidfVectorizer :
[0.6780689  0.68753033 0.67564289 0.68680252 0.67540029]
Moyenne Cross-validation : 0.6807


In [82]:
# Entraînemt final sur le meilleur modèle :
# Choisis le mdèle avec le meilleur score de validat° :
best_tfidf_model = LogisticRegression(max_iter=1000)
best_tfidf_model.fit(tfidf_vectorizer.fit_transform(X_train), y_train)

# Prédict° sur l'ensble de test :
y_pred_tfidf = best_tfidf_model.predict(tfidf_vectorizer.transform(X_test))

# Évaluat° finale sur l'ensble de test :
print("\nClassification Report pour TfidfVectorizer :")
print(classification_report(y_test, y_pred_tfidf))


Classification Report pour TfidfVectorizer :
              precision    recall  f1-score   support

    negative       0.72      0.59      0.65      1944
     neutral       0.63      0.74      0.68      2772
    positive       0.76      0.71      0.73      2154

    accuracy                           0.69      6870
   macro avg       0.70      0.68      0.69      6870
weighted avg       0.70      0.69      0.69      6870



In [107]:
# Évaluat° du mdèle final :
# Choisis le mdèle avec le meilleur score de validat° :
best_model = grid_search_tfidf.best_estimator_

# Prédict° sur l'ensble de test :
y_pred_tfidf = best_model.predict(X_test)

# Affiche le rapport de classificat° pr TfidfVectorizer :
print("\nClassification Report pour TfidfVectorizer :")
print(classification_report(y_test, y_pred_tfidf))


Classification Report pour TfidfVectorizer :
              precision    recall  f1-score   support

    negative       0.71      0.58      0.64      1944
     neutral       0.64      0.72      0.68      2772
    positive       0.74      0.74      0.74      2154

    accuracy                           0.69      6870
   macro avg       0.70      0.68      0.69      6870
weighted avg       0.69      0.69      0.69      6870

